In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Parsing



In [ ]:
!pip install demoparser2
!pip install pandas numpy scipy scikit-learn

In [ ]:
import os
import demoparser2
import pandas as pd
from tqdm import tqdm

Parse Per Demo

In [ ]:
def parse_single_demo_file(file_path, label, output_dir):
  parser = DemoParser(file_path)
  player_death_events = parser.parse_event("player_death")
  df = parser.parse_ticks(["pitch", "yaw", "tick", "steamid"])
  df['label'] = label
  base_dir = os.path.join(output_dir, "cheater" if label == 1 else "legit")
  os.makedirs(base_dir, exist_ok=True)
  for idx, event in tqdm(player_death_events.iterrows(), total=player_death_events.shape[0], desc=f"Parsing {os.path.basename(file_path)}"):
    start_tick = event["tick"] - 300
    end_tick = event["tick"]
    attacker = event.get("attacker_steamid")
    if attacker is not None:
      attacker_int = int(attacker)
      subdf = df[(df["tick"].between(start_tick, end_tick)) & (df["steamid"] == attacker_int)]
      if not subdf.empty:
        user_dir = os.path.join(base_dir, f"user_{attacker}")
        os.makedirs(user_dir, exist_ok=True)
        filename = f"kill_event_{start_tick}_to_{end_tick}.csv"
        file_path = os.path.join(user_dir, filename)
        subdf.to_csv(file_path, index=False)
        print(f"Data saved to {file_path}")

Parse Per Folder

In [ ]:
def parse_demo_folder(input_dir, label, output_dir):
    subfolder = 'cheater' if label == 1 else 'legit'
    base_dir = os.path.join(output_dir, subfolder)
    os.makedirs(base_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        if not filename.endswith('.dem'):
            continue
        demo_path = os.path.join(input_dir, filename)
        demo_base = os.path.splitext(filename)[0]

        # Skip if we’ve already parsed this demo
        already = [f for f in os.listdir(base_dir) if f.startswith(demo_base)]
        if already:
            print(f"Skipping {filename}, already parsed.")
            continue

        print(f"Parsing {filename}…")
        parser = DemoParser(demo_path)
        events = parser.parse_event("player_death")
        df = parser.parse_ticks(["pitch", "yaw", "tick", "steamid"])
        df['label'] = label

        for _, event in events.iterrows():
            start_tick = event["tick"] - 300
            end_tick   = event["tick"]
            attacker   = event.get("attacker_steamid")
            if not attacker:
                continue

            attacker_int = int(attacker)
            window = df[
                df["tick"].between(start_tick, end_tick) &
                (df["steamid"] == attacker_int)
            ]
            if window.empty:
                continue

            # 1) Drop any duplicate tick rows
            window = window.drop_duplicates(subset="tick")
            # 2) Reindex so we have exactly start_tick…end_tick‑1
            full_index = list(range(start_tick, end_tick))
            window = (window
                      .set_index("tick")
                      .reindex(full_index)
                      .fillna(method="ffill")  # carry last known pitch/yaw
                      .reset_index()
                      .rename(columns={"index": "tick"}))
            window['steamid'] = attacker_int
            window['label']   = label

            # Save
            user_dir = os.path.join(base_dir, f"user_{attacker}")
            os.makedirs(user_dir, exist_ok=True)
            csv_name  = f"{demo_base}_kill_{start_tick}_to_{end_tick}.csv"
            csv_path  = os.path.join(user_dir, csv_name)
            window.to_csv(csv_path, index=False)
            print(f"  → Saved {csv_path}")

In [ ]:
cheater_dir = '/content/drive/MyDrive/Skripsi/data/raw/demos/cheater'
legit_dir = '/content/drive/MyDrive/Skripsi/data/raw/demos/legit'
output_dir = '/content/drive/MyDrive/Skripsi/data/interim/parsed_csv'

In [ ]:
print("Parsing cheater demos...")
parse_demo_folder(cheater_dir, label=1, output_dir=output_dir)

print("Parsing legit demos...")
parse_demo_folder(legit_dir, label=0, output_dir=output_dir)

Parsing cheater demos...
Parsing thesis_test_cheat.dem…
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/cheater/user_76561199847502271/thesis_test_cheat_kill_311_to_611.csv


<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is dep

  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/cheater/user_76561199847502271/thesis_test_cheat_kill_513_to_813.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/cheater/user_76561199847502271/thesis_test_cheat_kill_707_to_1007.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/cheater/user_76561199847502271/thesis_test_cheat_kill_873_to_1173.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/cheater/user_76561199847502271/thesis_test_cheat_kill_1075_to_1375.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/cheater/user_76561199847502271/thesis_test_cheat_kill_1267_to_1567.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/cheater/user_76561199847502271/thesis_test_cheat_kill_1442_to_1742.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/cheater/user_76561199847502271/thesis_test_cheat_kill_1604_to_1904.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interi

<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is dep

  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/cheater/user_76561199847502271/thesis_test_cheat_kill_2921_to_3221.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/cheater/user_76561199847502271/thesis_test_cheat_kill_3154_to_3454.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/cheater/user_76561199847502271/thesis_test_cheat_kill_3277_to_3577.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/cheater/user_76561199847502271/thesis_test_cheat_kill_3447_to_3747.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/cheater/user_76561199847502271/thesis_test_cheat_kill_3621_to_3921.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/cheater/user_76561199847502271/thesis_test_cheat_kill_3772_to_4072.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/cheater/user_76561199847502271/thesis_test_cheat_kill_3892_to_4192.csv
  → Saved /content/drive/MyDrive/Skripsi/data/in

...

<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is dep

  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/legit/user_76561199039565051/Copy of cheater(xxx)_kill_41215_to_41515.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/legit/user_76561198822643426/Copy of cheater(xxx)_kill_42306_to_42606.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/legit/user_76561199470374288/Copy of cheater(xxx)_kill_44934_to_45234.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/legit/user_76561198822643426/Copy of cheater(xxx)_kill_45315_to_45615.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/legit/user_76561199469944110/Copy of cheater(xxx)_kill_45378_to_45678.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/legit/user_76561199469944110/Copy of cheater(xxx)_kill_46391_to_46691.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/legit/user_76561198822643426/Copy of cheater(xxx)_kill_46603_to_46903.csv
  → Saved /content/drive/My

<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is dep

  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/legit/user_76561198389634571/Copy of cheater(xxx)_kill_53773_to_54073.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/legit/user_76561199039565051/Copy of cheater(xxx)_kill_53965_to_54265.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/legit/user_76561199039565051/Copy of cheater(xxx)_kill_54152_to_54452.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/legit/user_76561199039565051/Copy of cheater(xxx)_kill_54475_to_54775.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/legit/user_76561198410519875/Copy of cheater(xxx)_kill_55282_to_55582.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/legit/user_76561199039565051/Copy of cheater(xxx)_kill_56241_to_56541.csv
  → Saved /content/drive/MyDrive/Skripsi/data/interim/parsed_csv/legit/user_76561199469944110/Copy of cheater(xxx)_kill_56377_to_56677.csv
  → Saved /content/drive/My

<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
<ipython-input-30-58e115666bc3>:46: FutureWarning: DataFrame.fillna with 'method' is dep